In [40]:
import pandas as pd
import numpy as np

# Step 1: Load the Data

In [42]:

titles_df = pd.read_csv('./datasets/titles.csv')
credits_df = pd.read_csv('./datasets/credits.csv')
print("******** Heads of the datasets  **************\n")
print(titles_df.head())
print(credits_df.head())

******** Heads of the datasets  **************

         id                                title   type  \
0  ts300399  Five Came Back: The Reference Films   SHOW   
1   tm84618                          Taxi Driver  MOVIE   
2  tm154986                          Deliverance  MOVIE   
3  tm127384      Monty Python and the Holy Grail  MOVIE   
4  tm120801                      The Dirty Dozen  MOVIE   

                                         description  release_year  \
0  This collection includes 12 World War II-era p...          1945   
1  A mentally unstable Vietnam War veteran works ...          1976   
2  Intent on seeing the Cahulawassee River before...          1972   
3  King Arthur, accompanied by his squire, recrui...          1975   
4  12 American military prisoners in World War II...          1967   

  age_certification  runtime                                       genres  \
0             TV-MA       51                            ['documentation']   
1                 R   

# Step 2: Initial Data Exploration

In [44]:

print("\n\n********* some information of the dataset ***********\n")
print(titles_df.info())
print(credits_df.info())
print("\n\n*****************      Null values ****************\n")
print(titles_df.isnull().sum())
print(credits_df.isnull().sum())
print("\n\n******** Summary statistics *****************\n")
print(titles_df.describe())
print(credits_df.describe())



********* some information of the dataset ***********

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5850 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5850 non-null   object 
 1   title                 5849 non-null   object 
 2   type                  5850 non-null   object 
 3   description           5832 non-null   object 
 4   release_year          5850 non-null   int64  
 5   age_certification     3231 non-null   object 
 6   runtime               5850 non-null   int64  
 7   genres                5850 non-null   object 
 8   production_countries  5850 non-null   object 
 9   seasons               2106 non-null   float64
 10  imdb_id               5447 non-null   object 
 11  imdb_score            5368 non-null   float64
 12  imdb_votes            5352 non-null   float64
 13  tmdb_popularity       5759 non-null   float64
 14  tmdb_score     

# Step 3: Handle missing values

## 3.1-missing values in titles_df


for the columns title and description the type of missing is MCAR so we use Imupation

In [48]:
titles_df.fillna({'description':'No discription is available'}, inplace=True)

titles_df.fillna({'title':'Missing title'}, inplace=True)

print(titles_df['description'].isnull().sum())
print(titles_df['title'].isnull().sum())

0
0


for the messing values in age_certification may be related to the release_year  then the type of the missing is MAR => use the methode Multiple imputation

In [50]:
titles_df['age_certification'] = titles_df.groupby('release_year')['age_certification'].transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else "Unknown"))

In [51]:
print(titles_df['age_certification'].isnull().sum())
print(titles_df['age_certification'])

0
0       TV-MA
1           R
2           R
3          PG
4           R
        ...  
5845    TV-MA
5846    TV-MA
5847    TV-MA
5848    PG-13
5849    TV-MA
Name: age_certification, Length: 5850, dtype: object


For the missing values in seasons columns, we notice that number of MOVIE samples is equal to number of missing values in Seasons columns.
Then, for all the samples with type==movie seasons<-0 instead of NaN

In [53]:
print("Is number of MOVIE samples equal to number of missing values in Seasons columns? => ", np.sum((titles_df['type']=='MOVIE'))==titles_df.isnull().sum()['seasons'])


Is number of MOVIE samples equal to number of missing values in Seasons columns? =>  True


In [54]:
titles_df['seasons']=np.where(
    (titles_df['seasons'].isna()) & (titles_df['type']=='MOVIE'), 
    0, 
    titles_df['seasons']
)

In [55]:
titles_df.isnull().sum()['seasons']

0

For the column imdb_id, we will impute the missing values with a unique number for each missing value because the column is an ID.

In [57]:
# Find the indices where values are missing
missing_indices = titles_df['imdb_id'].isna()

# Generate unique values starting from the maximum value in the column + 1

existing_ids=titles_df['imdb_id'].dropna()
max_id=max( (int(id[2:])) for id in existing_ids) 

# Generate unique IDs for missing values
new_ids = [f'tt{max_id + i + 1}' for i in range(missing_indices.sum())]

# Fill missing IDs with unique values
titles_df.loc[missing_indices, 'imdb_id'] = new_ids

#check missing values 
print(titles_df['imdb_id'].isnull().sum())

0


For the numerical columns imdb_score, 
imdb_vote, 
tmdb_populary, 1
tmdb_scowe will impute the missing values using a predictor KNN
Where the value to use as an imputer is predicted. 
To be more specific, we will use k-Nearest Neighbors imputer based on knn predictor. 

 > Each sample’s missing values are imputed using the mean value from n_neighbors nearest neighbors found in the training set. Two samples are close if the features that neither is missing are close.
311

In [59]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)
titles_df[['imdb_score', 'imdb_votes', 'tmdb_popularity', 'tmdb_score']] = imputer.fit_transform(titles_df[['imdb_score', 'imdb_votes', 'tmdb_popularity', 'tmdb_score']])

In [60]:
print(titles_df['imdb_score'].isnull().sum())

0


## 3.2-missing values in credits_df
There is only one column with missing values, character column, and the missing type is CMAR, we will fill the NaN values with "character unknown".

In [62]:
credits_df.fillna({'character':'character unknown'}, inplace=True)

# Step 4: Remove duplicates.

In [64]:
print(titles_df.duplicated('id').sum())
print(credits_df.duplicated('person_id').sum())



0
23212


We have duplicated values in the credits dataset 

In [133]:
credits_df.drop_duplicates('person_id',inplace=True)

In [135]:
print(credits_df.duplicated('person_id').sum())


0


# Step 5: Convert data types.

In [68]:
titles_df['release_year'] = pd.to_numeric(titles_df['release_year'], errors='coerce').astype(int)
titles_df['runtime'] = pd.to_numeric(titles_df['runtime'], errors='coerce').astype(int)
titles_df['seasons'] = pd.to_numeric(titles_df['seasons'], errors='coerce').astype(int)
titles_df['imdb_score'] = titles_df['imdb_score'].astype(float)
titles_df['tmdb_score'] = titles_df['tmdb_score'].astype(float)
titles_df['imdb_votes'] = titles_df['imdb_votes'].astype(float)
titles_df['tmdb_popularity'] = titles_df['tmdb_popularity'].astype(float)

# Step 6: Creating new relations for a good database modelization

We notice that in one attributs there is a list of values and this not a good practice in database conception

In [71]:
print(titles_df['genres'])

0                                 ['documentation']
1                                ['drama', 'crime']
2       ['drama', 'action', 'thriller', 'european']
3                   ['fantasy', 'action', 'comedy']
4                                 ['war', 'action']
                           ...                     
5845                           ['romance', 'drama']
5846                                      ['drama']
5847                                     ['comedy']
5848                                             []
5849              ['family', 'animation', 'comedy']
Name: genres, Length: 5850, dtype: object


In [72]:
print(titles_df['production_countries'])

0             ['US']
1             ['US']
2             ['US']
3             ['GB']
4       ['GB', 'US']
            ...     
5845          ['NG']
5846              []
5847          ['CO']
5848          ['US']
5849              []
Name: production_countries, Length: 5850, dtype: object


## 6.1-Creating a Genre table (data frame => csv file)

In [74]:
#this let us to transform the list of object stored as a sting into a real list 
import ast
titles_df['genres'].apply(ast.literal_eval)

0                           [documentation]
1                            [drama, crime]
2       [drama, action, thriller, european]
3                 [fantasy, action, comedy]
4                             [war, action]
                       ...                 
5845                       [romance, drama]
5846                                [drama]
5847                               [comedy]
5848                                     []
5849            [family, animation, comedy]
Name: genres, Length: 5850, dtype: object

In [75]:
import ast
titles_df['genres']=titles_df['genres'].apply(ast.literal_eval)
all_genres = titles_df['genres'].explode() #we extract all the genres 
print(all_genres)

0       documentation
1               drama
1               crime
2               drama
2              action
            ...      
5847           comedy
5848              NaN
5849           family
5849        animation
5849           comedy
Name: genres, Length: 15147, dtype: object


In [76]:
all_genres.unique() #we keep unique values of genres 

array(['documentation', 'drama', 'crime', 'action', 'thriller',
       'european', 'fantasy', 'comedy', 'war', 'romance', 'western',
       'history', 'music', 'family', 'horror', 'scifi', 'animation',
       'reality', 'sport', nan], dtype=object)

In [77]:
unique_genres = pd.DataFrame(all_genres.dropna().unique(), columns=['genre_name'])
print(unique_genres)

       genre_name
0   documentation
1           drama
2           crime
3          action
4        thriller
5        european
6         fantasy
7          comedy
8             war
9         romance
10        western
11        history
12          music
13         family
14         horror
15          scifi
16      animation
17        reality
18          sport


In [78]:
# Save to CSV
unique_genres.to_csv('./datasets/genres.csv', index=False)


## 6.2-Creating a ProductionCountry table (data frame => csv file)

In [80]:
import ast
titles_df['production_countries']=titles_df['production_countries'].apply(ast.literal_eval)
all_countries = titles_df['production_countries'].explode() #we extract all the countries
print(all_countries)
all_countries.unique() #we keep unique values of countries
unique_countries = pd.DataFrame(all_countries.dropna().unique(), columns=['country'])
print(unique_countries)
# Save to CSV
unique_countries.to_csv('./datasets/countries.csv', index=False)

0        US
1        US
2        US
3        GB
4        GB
       ... 
5845     NG
5846    NaN
5847     CO
5848     US
5849    NaN
Name: production_countries, Length: 6757, dtype: object
    country
0        US
1        GB
2        EG
3        DE
4        IN
..      ...
104      AF
105      AO
106      NA
107      KN
108      FO

[109 rows x 1 columns]


## 6.3-Creating a TitleProdCountry table (data frame => csv file)

In [82]:
TitleProdCountry = pd.DataFrame(columns=['title_id', 'prodCountry'])
#Iterate through titles_df to populate TitleHasGenre
for index, row in titles_df.iterrows():
    # Get the list of genres directly
    prodCountries = row['production_countries']
    
    # Create a DataFrame for this title with one row per genre
    title_countries = pd.DataFrame({
        'title_id': [row['id']] * len(prodCountries),
        'prodCountry': prodCountries
    })
    
    # Append to TitleHasGenre DataFrame
    TitleProdCountry = pd.concat([TitleProdCountry, title_countries], ignore_index=True)

print(TitleProdCountry.head())
TitleProdCountry.to_csv('./datasets/titleProdCountry.csv', index=False)

   title_id prodCountry
0  ts300399          US
1   tm84618          US
2  tm154986          US
3  tm127384          GB
4  tm120801          GB


In [83]:
print(TitleProdCountry.head())

   title_id prodCountry
0  ts300399          US
1   tm84618          US
2  tm154986          US
3  tm127384          GB
4  tm120801          GB


## 6.4-Creating a TitleHasGenre table (data frame => csv file)

In [85]:
TitleHasGenre = pd.DataFrame(columns=['title_id', 'genre'])
#Iterate through titles_df to populate TitleHasGenre
for index, row in titles_df.iterrows():
    # Get the list of genres directly
    genres = row['genres']
    
    # Create a DataFrame for this title with one row per genre
    title_genres = pd.DataFrame({
        'title_id': [row['id']] * len(genres),
        'genre': genres
    })
    
    # Append to TitleHasGenre DataFrame
    TitleHasGenre = pd.concat([TitleHasGenre, title_genres], ignore_index=True)

TitleHasGenre.to_csv('./datasets/titleHasGenre.csv', index=False)

In [86]:
TitleHasGenre.head()

,title_id,genre
0,ts300399,documentation
1,tm84618,drama
2,tm84618,crime
3,tm154986,drama
4,tm154986,action


## 6.5-drop the genres and production_countries columns

In [88]:
titles_df = titles_df.drop(columns=['genres', 'production_countries'], errors='ignore')


In [89]:
#Step 7: Save the cleaned dataset

In [90]:
# Save cleaned data to new CSV files
titles_df.to_csv('./datasets/cleaned_titles.csv', index=False)
credits_df.to_csv('./datasets/cleaned_credits.csv', index=False)